In [ ]:
import os
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

In [ ]:
import base64
import json
import shutil
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform
from oauth2client.client import GoogleCredentials

In [ ]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
BUCKET = PROJECT + "-flowers"
FILE_DIR = f"gs://{BUCKET}/data"
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

CLASSES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

BATCH_SIZE = 32

In [ ]:
# !gsutil mb gs://{BUCKET}
# !gsutil cp gs://asl-public/data/flowers/tfrecords/* {FILE_DIR}

In [ ]:
!gsutil ls {FILE_DIR}

In [ ]:
TRAIN_PATTERN = FILE_DIR + "/train*"
EVAL_PATTERN = FILE_DIR + "/eval*"


def parse_example(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    example["image"] = tf.image.resize(
        example["image"], [IMG_HEIGHT, IMG_WIDTH]
    )
    example["image"] = example["image"] / 255
    return example["image"], example["label"]


train_ds = (
    tf.data.TFRecordDataset(tf.io.gfile.glob(TRAIN_PATTERN))
    .map(parse_example)
    .batch(BATCH_SIZE)
)
eval_ds = (
    tf.data.TFRecordDataset(tf.io.gfile.glob(EVAL_PATTERN))
    .map(parse_example)
    .batch(10)
)

In [ ]:
module_selection = "mobilenet_v2_100_224"
module_handle = "https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(
    module_selection
)

transfer_model = tf.keras.Sequential(
    [
        hub.KerasLayer(module_handle, trainable=True),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(
            len(CLASSES),
            activation="softmax",
            kernel_regularizer=tf.keras.regularizers.l2(0.0001),
        ),
    ]
)

transfer_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
transfer_model.fit(
    train_ds,
    epochs=5,
    validation_data=eval_ds,
)

In [ ]:
shutil.rmtree("export", ignore_errors=True)
os.mkdir("export")
transfer_model.save("export/flowers_model")

In [ ]:
!ls export/flowers_model

---

## Build a streamlit application

### Streamlit app file

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model

st.set_page_config(page_title='5-Flower Classifier', page_icon='🌷')

st.title('5-Flower Classifier')

st.markdown("Welcome to this simple web application that classifies 5 flowers (daisy, dandelion, roses, sunflowers, tulips).")

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
CLASSES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

# Cache model so the app doesn't need to load the model from the second prediction.
@st.cache_resource(show_spinner=False)
def load_and_cache_model():
    model_path = "flowers_model"
    model = load_model(model_path)
    return model


def read_image(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def predict(model, image):
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    pred_index = np.argmax(predictions[0])
    return predictions[0][pred_index], CLASSES[pred_index]


def main():
    file_uploaded = st.file_uploader("Choose File", type=["png","jpg","jpeg"])
    if file_uploaded is not None:
        image = read_image(file_uploaded.read())
        st.image(image.numpy(), caption='Uploaded Image', use_column_width=True)
        class_btn = st.button("Classify")
        if class_btn:
            with st.spinner('Model predicting....'):
                global model
                model = load_and_cache_model()
                prob, prediction = predict(model, image)
                st.success(f"Prediction: {prediction} - {prob:.2%}")


if __name__ == "__main__":
    main()

### Define Dockerfile and dependencies

In [ ]:
%%writefile requirements.txt
tensorflow==2.12.0
numpy==1.23.5
streamlit==1.29.0

In [ ]:
%%writefile Dockerfile
FROM python:3.10.14

WORKDIR /app

COPY requirements.txt /app

RUN pip install -r requirements.txt

COPY export /app

COPY app.py /app

EXPOSE 8080

CMD streamlit run --server.port 8080 --server.enableCORS false app.py

### Build the container and push it to Artifact Registry

In [ ]:
STREAMLIT_ARTIFACT_REG_REPO = "flower-classification-app"
os.environ["STREAMLIT_ARTIFACT_REG_REPO"] = STREAMLIT_ARTIFACT_REG_REPO

In [ ]:
%%bash
if ! gcloud artifacts repositories describe $STREAMLIT_ARTIFACT_REG_REPO \
       --location=$REGION > /dev/null 2>&1; then
    gcloud artifacts repositories create $STREAMLIT_ARTIFACT_REG_REPO \
        --project=$PROJECT --location=$REGION --repository-format=docker
fi

#### Define config.yaml
use cache

In [ ]:
CONTAINER_PATH = (
    f"us-central1-docker.pkg.dev/{PROJECT}/{STREAMLIT_ARTIFACT_REG_REPO}/app"
)
os.environ["CONTAINER_PATH"] = CONTAINER_PATH

In [ ]:
%%bash
echo > ./config.yaml "steps:
- name: 'gcr.io/cloud-builders/docker'
  entrypoint: 'bash'
  args: ['-c', 'docker pull $CONTAINER_PATH:latest || exit 0']
- name: 'gcr.io/cloud-builders/docker'
  args: [
            'build',
            '-t', '$CONTAINER_PATH:latest',
            '--cache-from', '$CONTAINER_PATH:latest',
            '.'
        ]
images: ['$CONTAINER_PATH:latest']"

In [ ]:
!gcloud builds submit --config config.yaml --region $REGION .

### Deploy to Cloud Run

In [ ]:
APP_NAME = "flower-classification"
os.environ["APP_NAME"] = APP_NAME

In [ ]:
%%bash
echo 'Deploying the application to Cloud Run...'
gcloud run deploy $APP_NAME \
  --image $CONTAINER_PATH:latest --min-instances 1 --max-instances 1 --cpu 1 \
  --memory 4Gi --region us-central1 > /dev/null 2>&1 && \
echo 'Deployment Done.'

### Connect to Cloud Run app via Cloud Shell

Run the command below in Cloud Shell to proxy a service to localhost.

In [ ]:
print(
    f"gcloud run services proxy {APP_NAME} --project {PROJECT} --region {REGION}"
)

From Cloud Shell, click Web Preview -> Preview on Port 8080 and open the application.

### Prediction
Find a flower image (daisy, dandelion, roses, sunflowers or tulips) on Google Image Search, upload to the app, and try the prediction.

The first access and prediction may take some time, but it'll be faster from the second time thanks to the cache.